In [55]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.models import sequential
from tensorflow.python.keras.layers import LSTM,Embedding,Dense
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os
import numpy as np

In [37]:
FEATURES_FILE_DIR = './Dataset/features.txt'
ENG_VOCAB_FEATURES_FILE_DIR = './Dataset/eng_vocab_features.txt'
HAM_CLEAN_DIR = './Dataset/ham_clean/'
SPAM_CLEAN_DIR = './Dataset/spam_clean/'

In [7]:
tokenizer = Tokenizer(num_words=2341,oov_token="<OOV>")

In [8]:
with open(ENG_VOCAB_FEATURES_FILE_DIR) as file :
    line = file.readline()


In [9]:
tokenizer.fit_on_texts(line.split(" "))

In [10]:
tokenizer.word_index

{'<OOV>': 1,
 'number': 2,
 'list': 3,
 'get': 4,
 'one': 5,
 'free': 6,
 'time': 7,
 'new': 8,
 'message': 9,
 'people': 10,
 'please': 11,
 'information': 12,
 'address': 13,
 'would': 14,
 'make': 15,
 'click': 16,
 'like': 17,
 'business': 18,
 'use': 19,
 'company': 20,
 'money': 21,
 'work': 22,
 'also': 23,
 'order': 24,
 'year': 25,
 'name': 26,
 'service': 27,
 'want': 28,
 'site': 29,
 'day': 30,
 'need': 31,
 'web': 32,
 'send': 33,
 'state': 34,
 'system': 35,
 'program': 36,
 'report': 37,
 'may': 38,
 'way': 39,
 'see': 40,
 'right': 41,
 'receive': 42,
 'go': 43,
 'file': 44,
 'first': 45,
 'know': 46,
 'could': 47,
 'offer': 48,
 'home': 49,
 'product': 50,
 'even': 51,
 'world': 52,
 'user': 53,
 'take': 54,
 'mail': 55,
 'help': 56,
 'group': 57,
 'find': 58,
 'many': 59,
 'much': 60,
 'today': 61,
 'wrote': 62,
 'price': 63,
 'link': 64,
 'million': 65,
 'change': 66,
 'line': 67,
 'life': 68,
 'phone': 69,
 'best': 70,
 'say': 71,
 'month': 72,
 'government': 73,
 '

In [83]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index),1),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1,activation="sigmoid")    
])
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

In [84]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 1)           2342      
                                                                 
 lstm_4 (LSTM)               (None, 64)                16896     
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 19,303
Trainable params: 19,303
Non-trainable params: 0
_________________________________________________________________


In [39]:
tokenizer.texts_to_sequences([["number" ,'love', 'book',"dollar"],["number" ,'love', 'book',"dollar"]])

[[2, 491, 273, 96], [2, 491, 273, 96]]

In [40]:
def load_database():
    X=[]
    Y=[]
    #we start with the ham 
    for _ ,__ ,ham_files in os.walk(HAM_CLEAN_DIR):
        print("ham files found : ",len(ham_files))
    for file in ham_files :
        with open(HAM_CLEAN_DIR+file,encoding = "ISO-8859-1") as f:
           line = f.readlines()
        X.append(line[0].split(" "))
        Y.append(0)
    
    for _ ,__ ,spam_files in os.walk(SPAM_CLEAN_DIR):
        print("spam files found : ",len(spam_files))
    for file in spam_files :
        with open(SPAM_CLEAN_DIR+file,encoding = "ISO-8859-1") as f:
           line = f.readlines()
        X.append(line[0].split(" "))
        Y.append(1)
    
    X = tokenizer.texts_to_sequences(X)
    
    return X,Y

In [64]:
X,Y = load_database()

ham files found :  6951
spam files found :  2397


In [65]:
X = tf.keras.preprocessing.sequence.pad_sequences(X,2000)


In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [74]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [85]:
model.fit(X_train, y_train,batch_size =16, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
409/409 [==============================] - 47s 111ms/step - loss: 0.4910 - accuracy: 0.7856 - val_loss: 0.2547 - val_accuracy: 0.9105
Epoch 2/20
409/409 [==============================] - 45s 110ms/step - loss: 0.2022 - accuracy: 0.9363 - val_loss: 0.1984 - val_accuracy: 0.9358
Epoch 3/20
409/409 [==============================] - 46s 112ms/step - loss: 0.1636 - accuracy: 0.9460 - val_loss: 0.1720 - val_accuracy: 0.9344
Epoch 4/20
409/409 [==============================] - 45s 110ms/step - loss: 0.3344 - accuracy: 0.8984 - val_loss: 0.7398 - val_accuracy: 0.4988
Epoch 5/20
409/409 [==============================] - 45s 109ms/step - loss: 0.5491 - accuracy: 0.7406 - val_loss: 0.4862 - val_accuracy: 0.7683
Epoch 6/20
409/409 [==============================] - 46s 113ms/step - loss: 0.4384 - accuracy: 0.7937 - val_loss: 0.3581 - val_accuracy: 0.8335
Epoch 7/20
409/409 [==============================] - 45s 110ms/step - loss: 0.1950 - accuracy: 0.9300 - val_loss: 0.1492 - val_ac